### First few cells is just data preparation

My implementation is build on top of nequip and allegro python libraries to simplify data load and preprocessing.
The wigner simbols are from e3nn
Here are the links
https://github.com/mir-group/nequip
https://github.com/mir-group/allegro

In [4]:
# reference example
from nequip.data import dataset_from_config
from nequip.utils import Config
#from nequip.utils.misc import get_default_device_name
#from nequip.utils.config import _GLOBAL_ALL_ASKED_FOR_KEYS

from nequip.model import model_from_config


default_config = dict(
    root="./",
    tensorboard=False,
    wandb=False,
    model_builders=[
        "SimpleIrrepsConfig",
        "EnergyModel",
        "PerSpeciesRescale",
        "StressForceOutput",
        "RescaleEnergyEtc",
    ],
    dataset_statistics_stride=1,
    device='cpu',
    default_dtype="float64",
    model_dtype="float32",
    allow_tf32=True,
    verbose="INFO",
    model_debug_mode=False,
    equivariance_test=False,
    grad_anomaly_mode=False,
    gpu_oom_offload=False,
    append=False,
    warn_unused=False,
    _jit_bailout_depth=2,  # avoid 20 iters of pain, see https://github.com/pytorch/pytorch/issues/52286
    # Quote from eelison in PyTorch slack:
    # https://pytorch.slack.com/archives/CDZD1FANA/p1644259272007529?thread_ts=1644064449.039479&cid=CDZD1FANA
    # > Right now the default behavior is to specialize twice on static shapes and then on dynamic shapes.
    # > To reduce warmup time you can do something like setFusionStrartegy({{FusionBehavior::DYNAMIC, 3}})
    # > ... Although we would wouldn't really expect to recompile a dynamic shape fusion in a model,
    # > provided broadcasting patterns remain fixed
    # We default to DYNAMIC alone because the number of edges is always dynamic,
    # even if the number of atoms is fixed:
    _jit_fusion_strategy=[("DYNAMIC", 3)],
    # Due to what appear to be ongoing bugs with nvFuser, we default to NNC (fuser1) for now:
    # TODO: still default to NNC on CPU regardless even if change this for GPU
    # TODO: default for ROCm?
    _jit_fuser="fuser1",
)

# All default_config keys are valid / requested
#_GLOBAL_ALL_ASKED_FOR_KEYS.update(default_config.keys())

In [5]:
config = Config.from_file('./configs/example_ETN.yaml', defaults=default_config)
    

dataset = dataset_from_config(config, prefix="dataset")

validation_dataset = None

dataset[0]

AtomicData(atom_types=[21, 1], cell=[3, 3], edge_cell_shift=[364, 3], edge_index=[2, 364], forces=[21, 3], pbc=[3], pos=[21, 3], total_energy=[1])

In [6]:
# Trainer
from nequip.train.trainer import Trainer
from e3nn import o3

trainer = Trainer(model=None, **Config.as_dict(config))

# what is this
# to update wandb data?
config.update(trainer.params)

# = Train/test split =
trainer.set_dataset(dataset, validation_dataset)

# Some hyperparameteres
Nc = 10 # number of chennels for F features from ETN paper
N_rank_spec = 4 # hidden rank of reduction for type radial tensor
config['Nc'] = Nc
config['N_rank_spec'] = N_rank_spec

# ETN parameters
config['d'] = 4 # dimention of the tensor train
config['N_rank_ett'] = [4, 4, 4] # ranks of tensor train


# = Build model =
final_model = model_from_config(
    config=config, initialize=True, dataset=trainer.dataset_train
)

DEBUG:root:* Initialize Output
  ...generate file name results/aspirin/example/log
  ...open log file results/aspirin/example/log
  ...generate file name results/aspirin/example/metrics_epoch.csv
  ...open log file results/aspirin/example/metrics_epoch.csv
  ...generate file name results/aspirin/example/metrics_initialization.csv
  ...open log file results/aspirin/example/metrics_initialization.csv
  ...generate file name results/aspirin/example/metrics_batch_train.csv
  ...open log file results/aspirin/example/metrics_batch_train.csv
  ...generate file name results/aspirin/example/metrics_batch_val.csv
  ...open log file results/aspirin/example/metrics_batch_val.csv
  ...generate file name results/aspirin/example/best_model.pth
  ...generate file name results/aspirin/example/last_model.pth
  ...generate file name results/aspirin/example/trainer.pth
  ...generate file name results/aspirin/example/config.yaml
Torch device: cpu
instantiate Loss
...Loss_param = dict(
...   optional_args =

Replace string dataset_forces_rms to 31.252248764038086
Initially outputs are globally scaled by: 31.252248764038086, total_energy are globally shifted by None.
PerSpeciesScaleShift's arguments were in dataset units; rescaling:
  Original scales: [H: 31.252249, C: 31.252249, O: 31.252249] shifts: [H: -19318.355469, C: -19318.355469, O: -19318.355469]
  New scales: [H: 1.000000, C: 1.000000, O: 1.000000] shifts: [H: -618.142883, C: -618.142883, O: -618.142883]


In [7]:
import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
from allegro import with_edge_spin_length
from allegro import _keys
from torch import nn
import math

trainer.model = final_model

# Test configuration stores as dict of parameters
data0 = AtomicData.to_AtomicDataDict(dataset[0])

In [8]:
import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
    
from torch import nn
import math

# forward pass
data_new = final_model(data0)

### Main module

I'll skip the implementation of feature vector F because it heavily relies on nequip code for neighbor lists operations.
If you want you can look into the following files or ask me to add it to the notebook.

Otherwise look into this file

allegro/modules/ETN.py 

and corresponding layers


In [9]:
from typing import Optional, List, Tuple
import math
import functools

import torch
from torch import nn
from torch_runstats.scatter import scatter

from e3nn import o3
from e3nn.util.jit import compile_mode

from nequip.data import AtomicDataDict
from nequip.nn import GraphModuleMixin
from nequip.utils.tp_utils import tp_path_exists

from allegro.nn._fc import ScalarMLPFunction
from allegro import _keys

from allegro.nn._strided import Contracter_ETN
from allegro.nn.cutoffs import cosine_cutoff, polynomial_cutoff
from e3nn.o3 import wigner_3j



# Triangular ineguality for path existance
def tri_ineq(l1, l2, l3):
    return max([l1, l2, l3]) <= min([l1 + l2, l2 + l3, l1 + l3])


@compile_mode("script")
class ETN_Module_opt(nn.Module, GraphModuleMixin):
    def __init__(self,
                 d: int,
                 N_rank_ett: List[int], 
                 irreps_in=None,
                 out_field: str = AtomicDataDict.PER_ATOM_ENERGY_KEY):
        
        super().__init__()
        self.out_field = out_field
        
        
        self.d = d
        self.Nc = irreps_in[_keys.NODE_FEATURES_F][0][0]
        self.register_buffer("N_rank_ett", torch.as_tensor(N_rank_ett, dtype=torch.long))
        
        # set up irreps
        self._init_irreps(
            irreps_in=irreps_in,
            required_irreps_in=[
                _keys.NODE_FEATURES_F
            ],
            irreps_out={_keys.NODE_FEATURES_ETN: o3.Irreps(
                    [(self.Nc, ir) for _, ir in irreps_in[_keys.NODE_FEATURES_F] ]),
                        out_field: o3.Irreps([(1, (0, 1))])}
        )
        
        
        # Parameters of the network
        
        # tensors for atomic features encoding
        lmax = irreps_in[_keys.EDGE_FEATURES_F].lmax # maximum spherical harmonic
        self.lmax = lmax
        
        # Second order cores(first and last)
        self.core2_1 = torch.nn.Parameter(torch.Tensor(lmax+1, self.Nc, N_rank_ett[0]))
        self.core2_d = torch.nn.Parameter(torch.Tensor(lmax+1, N_rank_ett[-1], self.Nc))
        
        
        
        # Third order cores
        # Assume irreps does not change 
        base_in1 = o3.Irreps([el[1] for el in irreps_in[_keys.EDGE_FEATURES_F]])
        base_in2 = o3.Irreps([el[1] for el in irreps_in[_keys.EDGE_FEATURES_F]])
        base_out = o3.Irreps([el[1] for el in irreps_in[_keys.EDGE_FEATURES_F]])
        

        # Building instructions
        instructions: List[Tuple[int, int, int]] = []
        tmp_i_out: int = 0
        for i_out, (_, ir_out) in enumerate(base_out):
            for i_1, (_, ir_in1) in enumerate(base_in1):
                for i_2, (_, ir_in2) in enumerate(base_in2):
                    if ir_out in ir_in1 * ir_in2:
                        instructions.append((i_1, i_2, i_out))
        
                        tmp_i_out += 1

        self.instructions = instructions
        
        # building large w3j
        w3j_values = []
        w3j_index = []
        for i_in1, i_in2, i_out in instructions:
            mul_ir_in1 = base_in1[i_in1]
            mul_ir_in2 = base_in2[i_in2]
            mul_ir_out = base_out[i_out]
    
            assert mul_ir_in1.ir.p * mul_ir_in2.ir.p == mul_ir_out.ir.p
            assert (
                tri_ineq(mul_ir_in1.ir.l, mul_ir_in2.ir.l, mul_ir_out.ir.l)
            )
    
            if mul_ir_in1.dim == 0 or mul_ir_in2.dim == 0 or mul_ir_out.dim == 0:
                raise ValueError
    
            this_w3j = o3.wigner_3j(mul_ir_in1.ir.l, mul_ir_in2.ir.l, mul_ir_out.ir.l)
            this_w3j_index = this_w3j.nonzero()
            w3j_values.append(
                this_w3j[this_w3j_index[:, 0], this_w3j_index[:, 1], this_w3j_index[:, 2]]
            )
    
            
            this_w3j_index[:, 0] += base_in1[: i_in1].dim
            this_w3j_index[:, 1] += base_in2[: i_in2].dim
            this_w3j_index[:, 2] += base_out[: i_out].dim
            # Now need to flatten the index to be for [pk][ij]
            w3j_index.append(
                torch.cat(
                    (   this_w3j_index[:, 2].unsqueeze(-1),
                        this_w3j_index[:, 0].unsqueeze(-1) * base_in2.dim
                        + this_w3j_index[:, 1].unsqueeze(-1),
                    ),
                    dim=1,
                )
            )
    
        num_paths: int = len(instructions)
    
        w3j = torch.sparse_coo_tensor(
            indices=torch.cat(w3j_index, dim=0).t(),
            values=torch.cat(w3j_values, dim=0),
            size=(
                num_paths * base_out.dim,
                base_in1.dim * base_in2.dim,
            ),
        ).coalesce()
        
        # in dense, must shape it for einsum:
        kij_shape = (
            base_out.dim,
            base_in1.dim,
            base_in2.dim,
        )
        
        # save to buffer in sparce mode + shape
        self.register_buffer("w3j", w3j)
        self.w3j_shape = (num_paths, ) + kij_shape
        
        # third order free parameters
        self.cores3 = [torch.nn.Parameter(torch.Tensor(N_rank_ett[r], self.Nc, N_rank_ett[r+1], num_paths)).to(w3j.device) for r in range(d - 2)] 
        
        self.reset_parameters()


        print(w3j.device)
        # Register layers
        self.tps = [Contracter_ETN(base_in1, 
                                   N_rank_ett[r], 
                                   base_in2, 
                                   self.Nc, 
                                   base_out, 
                                   N_rank_ett[r+1], 
                                   num_paths) for r in range(self.d - 2)]

        
        
    def forward(self, data: AtomicDataDict.Type) -> AtomicDataDict.Type:
        
        # Input features
        F = data[_keys.NODE_FEATURES_F]
        
        # Defining tensors for TorchScript
        u_out = torch.zeros((F.shape[0], F.shape[1], self.N_rank_ett[-1]), dtype=F.dtype,
            device=F.device) # temporary verctor output of etn
            
        
        data[_keys.NODE_FEATURES_ETN] = torch.zeros_like(F, dtype=F.dtype,
            device=F.device) # final feature output
        
        slices = self.irreps_in[AtomicDataDict.EDGE_ATTRS_KEY].slices() # slices over irreps

        # getting w3j in dense mode
        w3j_dense = (
            self.w3j.to_dense()
            .reshape(self.w3j_shape)
            .contiguous()
        )   
        
        # First transform using second order tensors
        for i, slice in enumerate(slices):
            u_out[:, slice, :] = torch.einsum('ij,Nmj->Nmi', self.core2_d[i], F[:, slice, :])

        # Series third order tensors
        for i in range(self.d - 2 - 1, -1, -1):

            # big contruction
            u_out = self.tps[i](u_out, F, w3j_dense, self.cores3[i].to(F.device))

        # Last transform using second order tensor
        for i, slice in enumerate(slices):
            data[_keys.NODE_FEATURES_ETN][:, slice, :] = torch.einsum('ij,Nmj->Nmi', self.core2_1[i], u_out[:, slice, :])
        
        
        # Reduction to scalar
        data[self.out_field] = (( data[_keys.NODE_FEATURES_ETN] * F ).sum(dim = (-2, -1) )).unsqueeze(-1)
        

        return data
    
    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.core2_1, a=math.sqrt(3))
        torch.nn.init.kaiming_uniform_(self.core2_d, a=math.sqrt(3))
        
        for i in range(len(self.cores3)):
            torch.nn.init.kaiming_uniform_(self.cores3[i], a=math.sqrt(3))

### Contruction of order 3 tensors

In [10]:
Nc = [10, 10, 10, 10]
rank = [1, 4, 4, 4, 1]
d = 4

base_in1 = o3.Irreps('1x0e + 1x1o + 2x2e')
base_in2 = o3.Irreps('1x0e + 1x1o + 2x2e')
base_out = o3.Irreps('1x0e + 1x1o + 2x2e')

# Building instructions
instructions: List[Tuple[int, int, int]] = []
tmp_i_out: int = 0
for i_out, (_, ir_out) in enumerate(base_out):
    for i_1, (_, ir_in1) in enumerate(base_in1):
        for i_2, (_, ir_in2) in enumerate(base_in2):
            if ir_out in ir_in1 * ir_in2:
                instructions.append((i_1, i_2, i_out))

                tmp_i_out += 1


# building large w3j
w3j_values = []
w3j_index = []
for i_in1, i_in2, i_out in instructions:
    mul_ir_in1 = base_in1[i_in1]
    mul_ir_in2 = base_in2[i_in2]
    mul_ir_out = base_out[i_out]

    assert mul_ir_in1.ir.p * mul_ir_in2.ir.p == mul_ir_out.ir.p
    assert (
        tri_ineq(mul_ir_in1.ir.l, mul_ir_in2.ir.l, mul_ir_out.ir.l)
    )

    if mul_ir_in1.dim == 0 or mul_ir_in2.dim == 0 or mul_ir_out.dim == 0:
        raise ValueError

    this_w3j = o3.wigner_3j(mul_ir_in1.ir.l, mul_ir_in2.ir.l, mul_ir_out.ir.l)
    this_w3j_index = this_w3j.nonzero()
    w3j_values.append(
        this_w3j[this_w3j_index[:, 0], this_w3j_index[:, 1], this_w3j_index[:, 2]]
    )


    this_w3j_index[:, 0] += base_in1[: i_in1].dim
    this_w3j_index[:, 1] += base_in2[: i_in2].dim
    this_w3j_index[:, 2] += base_out[: i_out].dim
    # Now need to flatten the index to be for [pk][ij]
    w3j_index.append(
        torch.cat(
            (   this_w3j_index[:, 2].unsqueeze(-1),
                this_w3j_index[:, 0].unsqueeze(-1) * base_in2.dim
                + this_w3j_index[:, 1].unsqueeze(-1),
            ),
            dim=1,
        )
    )

num_paths: int = len(instructions)

w3j = torch.sparse_coo_tensor(
    indices=torch.cat(w3j_index, dim=0).t(),
    values=torch.cat(w3j_values, dim=0),
    size=(
        num_paths * base_out.dim,
        base_in1.dim * base_in2.dim,
    ),
).coalesce()

# in dense, must shape it for einsum:
kij_shape = (
    base_out.dim,
    base_in1.dim,
    base_in2.dim,
)




In [11]:
cores3 = [torch.randn(num_paths, rank[r], Nc[r], rank[r+1]) for r in range(d)] 

In [12]:
[print(el.shape) for el in cores3]

torch.Size([11, 1, 10, 4])
torch.Size([11, 4, 10, 4])
torch.Size([11, 4, 10, 4])
torch.Size([11, 4, 10, 1])


[None, None, None, None]

In [13]:
instructions

[(0, 0, 0),
 (1, 1, 0),
 (2, 2, 0),
 (0, 1, 1),
 (1, 0, 1),
 (1, 2, 1),
 (2, 1, 1),
 (0, 2, 2),
 (1, 1, 2),
 (2, 0, 2),
 (2, 2, 2)]

In [14]:
# Left orthogonalize cores 1, 2

l = 5

instr_left = [el for i, el in enumerate(instructions) if el[-1] == 2]

ind_left = [i for i, el in enumerate(instructions) if el[-1] == 2]
ind_right = [i for i, el in enumerate(instructions) if el[0] == 2]

In [15]:
supercore_left = cores3[1][ind_left].flatten(0, -2)
supercore_right =torch.stack([cores3[2][ind, :, ...] for ind in ind_right], dim = -3).flatten(1)

supercore_left.shape, supercore_right.shape

(torch.Size([160, 4]), torch.Size([4, 160]))

In [16]:
instr_right

NameError: name 'instr_right' is not defined

In [ ]:
Q, R = torch.linalg.qr(supercore_left)

Q.shape, R.shape

In [ ]:
Q.T @ Q

In [ ]:
Q = Q.view((len(ind_left), 4, 10, 4))

Q.shape

In [ ]:
from e3nn.o3 import wigner_3j

In [ ]:
ins = instr_left[3]

ins

In [ ]:
torch.einsum('ijk,ijl->kl', wigner_3j(*ins), wigner_3j(*ins))*(2*l + 1)

In [18]:
def QR(mat):
    """
    Compute the QR decomposition. Backend can be changed.

    Parameters
    ----------
    mat : tn array
        DESCRIPTION.

    Returns
    -------
    Q : the Q matrix
    R : the R matrix

    """
    Q,R = tn.linalg.qr(mat)
    return Q, R

def lr_orthogonal(tt_cores, R, is_ttm, no_gpu = False):
    """
    Orthogonalize the TT-cores left to right.

    Parameters
    ----------
    tt_cores : list of torch tensors.
        The TT-cores as a list.

    Returns
    -------
    tt_cores : list of torch tensors.
        The orthogonal TT-cores as a list.

    """  
    
    d = len(tt_cores)

    rank_next = R[0]
    
    core_now = tt_cores[0]
    cores_new = d*[None]
    for i in range(d-1):
        
               
        if is_ttm:
            mode_shape = [core_now.shape[1],core_now.shape[2]]
            core_now = tn.reshape(core_now,[core_now.shape[0]*core_now.shape[1]*core_now.shape[2],-1])
        else:
            mode_shape = [core_now.shape[1]]
            core_now = tn.reshape(core_now,[core_now.shape[0]*core_now.shape[1],-1])
            
        # perform QR
        Qmat, Rmat = QR(core_now)
        core_now = Qmat
             
        # take next core
        core_next = tt_cores[i+1]
        shape_next = list(core_next.shape[1:])
        core_next = tn.reshape(core_next,[core_next.shape[0],-1])
        core_next = Rmat @ core_next
        core_next = tn.reshape(core_next,[core_now.shape[1]]+shape_next)
        
        # update the cores
        cores_new[i] = tn.reshape(core_now,[R[i]]+mode_shape+[-1])
        R[i+1] = core_now.shape[1]
        cores_new[i+1] = core_next
        
        core_now = core_next
        
        
    return cores_new, R

In [19]:
import torch as tn

def QR(mat):
    """
    Compute the QR decomposition. Backend can be changed.

    Parameters
    ----------
    mat : tn array
        DESCRIPTION.

    Returns
    -------
    Q : the Q matrix
    R : the R matrix

    """
    Q,R = tn.linalg.qr(mat)
    return Q, R

def lr_orthogonal(tt_cores, R, instr):
    """Make cores left orthogonal in equivariant sence
        tt_cores = [torch.randn(num_paths, rank[r], Nc[r], rank[r+1]) for r in range(d)] 
        R - old ranks
        instr - list of instr
    """
    lmax = max([el[2] for el in instr])
    ind_left = [[i for i, el in enumerate(instr) if el[-1] == l] for l in range(lmax+1)]
    ind_right = [[i for i, el in enumerate(instr) if el[0] == l] for l in range(lmax+1)]
    
    d = len(tt_cores)

    rank_next = R[0]
    
    core_now = tt_cores[0]
    cores_new = d*[None]
    
    
    # Loop over cores
    for i in range(d-1):
        # Loop over orthogonalized momentum
        
        core_next = tt_cores[i+1]
        shape_next = list(core_next.shape[2:]) 
        for l in range(lmax+1):
            #print(core_now.shape)
            mode_shape = [core_now.shape[2]]
            
            core_now_tmp = tn.reshape(core_now[ind_left[l]],[len(ind_left[l])*core_now.shape[1]*core_now.shape[2],-1])
            
            #print(core_now_tmp.shape)
            # perform QR
            Qmat, Rmat = QR(core_now_tmp)
            core_now_tmp = Qmat
            
            
            #print(Qmat.shape)
            #print(Rmat.shape)
            # take next core 
            #print(len(ind_left[l]), len(ind_right[l]))
            
            core_next_tmp = torch.stack([core_next[ind, ...] for ind in ind_right[l]], dim = -3).flatten(1)
            core_next_tmp = Rmat @ core_next_tmp
            

            
            core_next_tmp = tn.reshape(core_next_tmp,[core_now_tmp.shape[1]] + [len(ind_right[l])] + shape_next)
            
            # update the cores
            if l == 0:
                cores_new[i] = tn.zeros([len(instr)] + [R[i]] + mode_shape + [core_now_tmp.shape[1]])
                cores_new[i + 1] = tn.zeros([len(instr)] + [core_now_tmp.shape[1]] + shape_next)
                
            cores_new[i][ind_left[l]] = tn.reshape(core_now_tmp,[len(ind_left[l])] + [R[i]]+mode_shape+[-1])
            R[i+1] = core_now_tmp.shape[1]
            # TODO: mb transpose works too
            #cores_new[i+1][ind_right[l]] = core_next_tmp.transpose(0, 1)
            cores_new[i+1][ind_right[l]] = torch.stack([core_next_tmp[:, i] for i in range(len(ind_right[l]))], dim = 0)

        core_now = core_next
      
    cores_new = [el for el in cores_new]
    return cores_new, R
    

def rl_orthogonal(tt_cores, R, instr):
    """
    Orthogonalize the TT-cores right to left.

    Parameters
    ----------
    tt_cores : list of torch tensors.
        The TT-cores as a list.

    Returns
    -------
    tt_cores : list of torch tensors.
        The orthogonal TT-cores as a list.

    """  
    
    lmax = max([el[2] for el in instr])
    ind_left = [[i for i, el in enumerate(instr) if el[-1] == l] for l in range(lmax+1)]
    ind_right = [[i for i, el in enumerate(instr) if el[0] == l] for l in range(lmax+1)]
    
    d = len(tt_cores)

    rank_next = R[0]
    
    core_now = tt_cores[0]
    cores_new = d*[None]
        
    
    
    cores_new = d*[None]
    cores_new[-1] = tt_cores[-1]+0
    for i in range(d-1,0,-1):
        
        core_next = tt_cores[i - 1]
        for l in range(lmax+1):
            mode_shape = [cores_new[i].shape[2]]
            core_now = (torch.stack([cores_new[i][ind, ...] for ind in ind_right[l]], dim = -3).flatten(1)).t()
        
        
            Qmat, Rmat = QR(core_now)
            rnew = Rmat.shape[0]
            
            # update current core
            cores_new_tmp = tn.reshape(Qmat.T, [rnew]+[len(ind_right[l])] + mode_shape + [-1])
            cores_new[i][ind_right[l]] = cores_new_tmp.transpose(0, 1)
            
            R[i] = cores_new[i].shape[1]
            
            
            # and the i-1 one
            mode_shape = [core_next.shape[2]]
    
            core_next_tmp = tn.reshape(core_next[ind_left[l]],[len(ind_left[l])*core_next.shape[1]*core_next.shape[2],-1])
            core_next_tmp = core_next_tmp @ Rmat.T
            
            if l == 0:
                cores_new[i - 1] = tn.zeros([len(instr)] + [core_next.shape[1]] + mode_shape + [cores_new[i].shape[1]])
            
            cores_new[i - 1][ind_left[l]] = tn.reshape(core_next_tmp, [len(ind_left[l])] + [core_next.shape[1]] + mode_shape + [-1])
        
    return cores_new, R





In [20]:
Nc = [10, 10, 10, 10]
rank = [1, 4, 4, 4, 1]
d = 4

base_in1 = o3.Irreps('1x0e + 1x1o + 1x2e + 1x3o + 1x4e + 1x5o')
base_in2 = o3.Irreps('1x0e + 1x1o + 1x2e + 1x3o + 1x4e + 1x5o')
base_out = o3.Irreps('1x0e + 1x1o + 1x2e + 1x3o + 1x4e + 1x5o')

# Building instructions
instructions: List[Tuple[int, int, int]] = []
tmp_i_out: int = 0
for i_out, (_, ir_out) in enumerate(base_out):
    for i_1, (_, ir_in1) in enumerate(base_in1):
        for i_2, (_, ir_in2) in enumerate(base_in2):
            if ir_out in ir_in1 * ir_in2:
                instructions.append((i_1, i_2, i_out))

                tmp_i_out += 1


lmax = max([el[2] for el in instructions])
ind_left = [[i for i, el in enumerate(instructions) if el[-1] == l] for l in range(lmax+1)]
ind_right = [[i for i, el in enumerate(instructions) if el[0] == l] for l in range(lmax+1)]
                
print(len(instructions))

69


In [21]:
Nc = [10, 10, 10]
rank = [1, 5, 5, 1]
d = len(Nc)

num_paths = len(instructions)

cores3 = [torch.randn(num_paths, rank[r], Nc[r], rank[r+1]) for r in range(d)] 

In [24]:
cores_new, R = lr_orthogonal(cores3, rank, instructions)

In [25]:
# Test on lr orthogonality
cores_new_left = cores_new[0]
cores_new_left.flatten(0, -2).T @ cores_new_left.flatten(0, -2)

tensor([[ 6.0000e+00, -6.7055e-08,  3.1665e-08,  1.0245e-08,  3.3528e-08],
        [-6.7055e-08,  6.0000e+00,  6.1467e-08,  3.7253e-08, -8.2306e-08],
        [ 3.1665e-08,  6.1467e-08,  6.0000e+00,  4.5635e-08, -2.7940e-08],
        [ 1.0245e-08,  3.7253e-08,  4.5635e-08,  6.0000e+00, -3.7253e-08],
        [ 3.3528e-08, -8.2306e-08, -2.7940e-08, -3.7253e-08,  6.0000e+00]])

In [3]:
cores_new, R = rl_orthogonal(cores3, rank, instructions)

NameError: name 'rl_orthogonal' is not defined

In [1]:
# Test on rl orthogonality
cores_new_left = cores_new[-2].transpose(0, 1)
cores_new_left.flatten(1) @ cores_new_left.flatten(1).T

NameError: name 'cores_new' is not defined

In [422]:
print(cores_new[0].shape)
print(cores_new[1].shape)
print(cores_new[2].shape)

torch.Size([69, 1, 10, 5])
torch.Size([69, 5, 10, 5])
torch.Size([69, 5, 10, 1])


In [424]:
cores_new[0][ind_left[4]].flatten(0, -2).t() @ (cores_new[0][ind_left[4]].flatten(0, -2))

tensor([[ 1.0000e+00,  3.7253e-09,  3.7253e-09, -7.4506e-09, -1.6764e-08],
        [ 3.7253e-09,  1.0000e+00,  0.0000e+00, -2.9802e-08,  1.4435e-08],
        [ 3.7253e-09,  0.0000e+00,  1.0000e+00, -5.2154e-08, -2.0489e-08],
        [-7.4506e-09, -2.9802e-08, -5.2154e-08,  1.0000e+00, -4.6566e-09],
        [-1.6764e-08,  1.4435e-08, -2.0489e-08, -4.6566e-09,  1.0000e+00]])

In [416]:
cores_new[1].flatten(0, -2).t() @ (cores_new[1].flatten(0, -2))

tensor([[ 6.0000e+00, -2.9802e-08, -2.7008e-08,  3.7253e-09, -9.3132e-09],
        [-2.9802e-08,  6.0000e+00, -1.8626e-09,  6.3330e-08, -5.6461e-09],
        [-2.7008e-08, -1.8626e-09,  6.0000e+00,  1.7695e-08, -2.3749e-08],
        [-4.4703e-08,  5.1223e-08,  1.5832e-08,  6.0000e+00,  4.6566e-09],
        [-3.5390e-08, -2.4505e-08, -1.3504e-08,  4.6566e-09,  6.0000e+00]])

In [417]:
torch.round(cores_new[2].flatten(0, -2).t() @ (cores_new[2].flatten(0, -2))).shape

torch.Size([1, 1])

In [263]:
R

[1, 5, 5, 1]